Nama : Siti Ma'rifatul Khoiriyah

Nim : 210411100007

Kelas : PPW A

# **Tugas 2 Preprocessing Dan TF IDF**

**PRE PROCESSING**



---
Preprocessing adalah proses membersihkan dan mempersiapkan data mentah agar siap digunakan oleh model machine learning. Ini meliputi penanganan data yang hilang, normalisasi, mengubah data kategori menjadi angka, dan membersihkan teks. Tujuannya agar data lebih mudah dipahami dan diolah oleh model untuk hasil yang lebih akurat, Berikut adalah beberapa langkah umum dalam pre-processing teks:


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

df = pd.read_csv("/content/drive/My Drive/PPW/report/tugas_ppw/databerita100.csv")
df.head()

,judul,tanggal,isi,kategori
0,Warung Tenda Jepang Ini Punya Chicken Katsu Cu...,"Minggu, 08 Sep 2024 12:00 WIB",Jakarta - Warung Jepang tendaan di Fatmawati i...,Makanan
1,10 Warmindo yang Punya Menu Mie Nyemek dan Mie...,"Minggu, 08 Sep 2024 11:00 WIB",Jakarta - Warung mie instan atau warmindo kian...,Makanan
2,3 Resep Gorengan Populer yang Renyah Untuk Cam...,"Minggu, 08 Sep 2024 10:30 WIB",Tentang Bahan Langkah Jakarta - Di akhir pekan...,Makanan
3,"5 Etiket Makan Buah Secara Formal, Ternyata Be...","Minggu, 08 Sep 2024 10:00 WIB",Jakarta - Etiket makan formal banyak dibagikan...,Makanan
4,Resep Ayam Ungkep Serai yang Sedap Berbumbu Re...,"Minggu, 08 Sep 2024 09:00 WIB",Tentang Bahan Langkah Jakarta - Selain ayam go...,Makanan


1. CLEANSING

Tahapan proses cleansing data merupakan tahap pembersihan kata dari atribut yang tidak berpengaruh terhadap hasil klasifikasi sentimen. Komponen dokumen review memiliki beberapa atribut tidak berpengaruh terhadap sentimen diataranya url, html, emoji, simbol, angka dan tanda baca (~!@#$%^&*{}<>:|). Atribut yang tidak berpengaruh tersebut kemudian akan dihapus dan akan digantikan dengan karakter spasi.atribut-atribut tersebut dihapus dari teks untuk menghindari kebisingan (noise) dalam data. Setelah penghapusan, elemen-elemen ini digantikan dengan karakter spasi agar struktur kalimat tetap terjaga. Dengan pembersihan ini, data menjadi lebih fokus pada kata-kata atau frasa yang benar-benar relevan untuk menentukan sentimen, sehingga meningkatkan akurasi model dalam memprediksi sentimen.

In [3]:
import re
import string
import nltk

def remove_url(ulasan):
  url = re.compile(r'https?://\S+|www\.S+')
  return url.sub(r'', ulasan)

def remove_html(ulasan):
  html = re.compile(r'<.#?>')
  return html.sub(r'', ulasan)

def remove_emoji(ulasan):
  emoji_pattern = re.compile("["
      u"\U0001F600-\U0001F64F"
      u"\U0001F300-\U0001F5FF"
      u"\U0001F680-\U0001F6FF"
      u"\U0001F1E0-\U0001F1FF""]+", flags=re.UNICODE)
  return emoji_pattern.sub(r'', ulasan)

def remove_numbers(ulasan):
  ulasan = re.sub(r'\d+', '', ulasan)
  return ulasan

def remove_symbols(ulasan):
  ulasan = re.sub(r'[^a-zA-Z0-9\s]', '', ulasan)
  return ulasan

df['cleansing'] = df['isi'].apply(lambda x: remove_url(x))
df['cleansing'] = df['cleansing'].apply(lambda x: remove_html(x))
df['cleansing'] = df['cleansing'].apply(lambda x: remove_emoji(x))
df['cleansing'] = df['cleansing'].apply(lambda x: remove_symbols(x))
df['cleansing'] = df['cleansing'].apply(lambda x: remove_numbers(x))

df.head(5)

,judul,tanggal,isi,kategori,cleansing
0,Warung Tenda Jepang Ini Punya Chicken Katsu Cu...,"Minggu, 08 Sep 2024 12:00 WIB",Jakarta - Warung Jepang tendaan di Fatmawati i...,Makanan,Jakarta Warung Jepang tendaan di Fatmawati in...
1,10 Warmindo yang Punya Menu Mie Nyemek dan Mie...,"Minggu, 08 Sep 2024 11:00 WIB",Jakarta - Warung mie instan atau warmindo kian...,Makanan,Jakarta Warung mie instan atau warmindo kian ...
2,3 Resep Gorengan Populer yang Renyah Untuk Cam...,"Minggu, 08 Sep 2024 10:30 WIB",Tentang Bahan Langkah Jakarta - Di akhir pekan...,Makanan,Tentang Bahan Langkah Jakarta Di akhir pekan ...
3,"5 Etiket Makan Buah Secara Formal, Ternyata Be...","Minggu, 08 Sep 2024 10:00 WIB",Jakarta - Etiket makan formal banyak dibagikan...,Makanan,Jakarta Etiket makan formal banyak dibagikan ...
4,Resep Ayam Ungkep Serai yang Sedap Berbumbu Re...,"Minggu, 08 Sep 2024 09:00 WIB",Tentang Bahan Langkah Jakarta - Selain ayam go...,Makanan,Tentang Bahan Langkah Jakarta Selain ayam gor...


2. CASE FOLDING

Pada tahap case folding huruf kapital pada semua dokumen ulasan diubah menjadi huruf kecil atau disebut lowercase. Hal ini bertujuan agar menghiangkan redudansi data yang hanya berbeda pada hurufnya saja. Misalnya, kata "Makanan" dan "makanan" dianggap sama dalam konteks analisis teks, tetapi tanpa case folding, keduanya akan dianggap berbeda oleh komputer. Dengan mengubah semua huruf menjadi kecil, kita menyatukan semua variasi penulisan menjadi satu bentuk yang konsisten, sehingga menghindari perhitungan yang berlebihan atau salah interpretasi data.

In [4]:
def case_folding(text):
    if isinstance(text, str):
      lowercase_text = text.lower()
      return lowercase_text
    else :
      return text

df ['case_folding'] = df['cleansing'].apply(case_folding)

df.head(5)

,judul,tanggal,isi,kategori,cleansing,case_folding
0,Warung Tenda Jepang Ini Punya Chicken Katsu Cu...,"Minggu, 08 Sep 2024 12:00 WIB",Jakarta - Warung Jepang tendaan di Fatmawati i...,Makanan,Jakarta Warung Jepang tendaan di Fatmawati in...,jakarta warung jepang tendaan di fatmawati in...
1,10 Warmindo yang Punya Menu Mie Nyemek dan Mie...,"Minggu, 08 Sep 2024 11:00 WIB",Jakarta - Warung mie instan atau warmindo kian...,Makanan,Jakarta Warung mie instan atau warmindo kian ...,jakarta warung mie instan atau warmindo kian ...
2,3 Resep Gorengan Populer yang Renyah Untuk Cam...,"Minggu, 08 Sep 2024 10:30 WIB",Tentang Bahan Langkah Jakarta - Di akhir pekan...,Makanan,Tentang Bahan Langkah Jakarta Di akhir pekan ...,tentang bahan langkah jakarta di akhir pekan ...
3,"5 Etiket Makan Buah Secara Formal, Ternyata Be...","Minggu, 08 Sep 2024 10:00 WIB",Jakarta - Etiket makan formal banyak dibagikan...,Makanan,Jakarta Etiket makan formal banyak dibagikan ...,jakarta etiket makan formal banyak dibagikan ...
4,Resep Ayam Ungkep Serai yang Sedap Berbumbu Re...,"Minggu, 08 Sep 2024 09:00 WIB",Tentang Bahan Langkah Jakarta - Selain ayam go...,Makanan,Tentang Bahan Langkah Jakarta Selain ayam gor...,tentang bahan langkah jakarta selain ayam gor...


3. TOKENIZATION

Tahap Tokenization merupakan pemotongan kata berdasarkan tiap kata yang menyusunnya menjadi potongan tunggal. Kata dalam dokumen yang dimaksud adalah kata yang dipisah oleh spasi, sehingga proses tokenisasi mengandalkan karakter spasi pada dokumen untuk melakukan pemisahan kata.
contoh, kalimat "Saya suka makan pizza" akan dipecah menjadi token ["Saya", "suka", "makan", "pizza"].

In [5]:
def tokenize(text):
    tokens = text.split()
    return tokens

df['tokenize'] = df['case_folding'].apply(tokenize)

df.head(5)

,judul,tanggal,isi,kategori,cleansing,case_folding,tokenize
0,Warung Tenda Jepang Ini Punya Chicken Katsu Cu...,"Minggu, 08 Sep 2024 12:00 WIB",Jakarta - Warung Jepang tendaan di Fatmawati i...,Makanan,Jakarta Warung Jepang tendaan di Fatmawati in...,jakarta warung jepang tendaan di fatmawati in...,"[jakarta, warung, jepang, tendaan, di, fatmawa..."
1,10 Warmindo yang Punya Menu Mie Nyemek dan Mie...,"Minggu, 08 Sep 2024 11:00 WIB",Jakarta - Warung mie instan atau warmindo kian...,Makanan,Jakarta Warung mie instan atau warmindo kian ...,jakarta warung mie instan atau warmindo kian ...,"[jakarta, warung, mie, instan, atau, warmindo,..."
2,3 Resep Gorengan Populer yang Renyah Untuk Cam...,"Minggu, 08 Sep 2024 10:30 WIB",Tentang Bahan Langkah Jakarta - Di akhir pekan...,Makanan,Tentang Bahan Langkah Jakarta Di akhir pekan ...,tentang bahan langkah jakarta di akhir pekan ...,"[tentang, bahan, langkah, jakarta, di, akhir, ..."
3,"5 Etiket Makan Buah Secara Formal, Ternyata Be...","Minggu, 08 Sep 2024 10:00 WIB",Jakarta - Etiket makan formal banyak dibagikan...,Makanan,Jakarta Etiket makan formal banyak dibagikan ...,jakarta etiket makan formal banyak dibagikan ...,"[jakarta, etiket, makan, formal, banyak, dibag..."
4,Resep Ayam Ungkep Serai yang Sedap Berbumbu Re...,"Minggu, 08 Sep 2024 09:00 WIB",Tentang Bahan Langkah Jakarta - Selain ayam go...,Makanan,Tentang Bahan Langkah Jakarta Selain ayam gor...,tentang bahan langkah jakarta selain ayam gor...,"[tentang, bahan, langkah, jakarta, selain, aya..."


4. STOPWORD REMOVAL

Dalam tahapan proses Stopword Removal kata yang tidak memiliki pengaruh signifikan dalam kalimat akan dihilangkan. Dalam pre processing ini penulis menghapus stopword pada data ulasan berdasar daftar kalimat stopword diantaranya yaitu “yang”, “dan”, “di”, “dari”, dll.
contoh, kalimat "Saya pergi ke pasar dan membeli buah" akan diubah menjadi "Saya pergi pasar membeli buah" setelah proses stopword removal.

In [6]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('indonesian')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
def remove_stopwords(text):
  return [word for word in text if word not in stop_words]

df['stopword_removal'] = df['tokenize'].apply(lambda x: ' '.join(remove_stopwords(x)))

df.head(5)

,judul,tanggal,isi,kategori,cleansing,case_folding,tokenize,stopword_removal
0,Warung Tenda Jepang Ini Punya Chicken Katsu Cu...,"Minggu, 08 Sep 2024 12:00 WIB",Jakarta - Warung Jepang tendaan di Fatmawati i...,Makanan,Jakarta Warung Jepang tendaan di Fatmawati in...,jakarta warung jepang tendaan di fatmawati in...,"[jakarta, warung, jepang, tendaan, di, fatmawa...",jakarta warung jepang tendaan fatmawati tujuan...
1,10 Warmindo yang Punya Menu Mie Nyemek dan Mie...,"Minggu, 08 Sep 2024 11:00 WIB",Jakarta - Warung mie instan atau warmindo kian...,Makanan,Jakarta Warung mie instan atau warmindo kian ...,jakarta warung mie instan atau warmindo kian ...,"[jakarta, warung, mie, instan, atau, warmindo,...",jakarta warung mie instan warmindo kian popule...
2,3 Resep Gorengan Populer yang Renyah Untuk Cam...,"Minggu, 08 Sep 2024 10:30 WIB",Tentang Bahan Langkah Jakarta - Di akhir pekan...,Makanan,Tentang Bahan Langkah Jakarta Di akhir pekan ...,tentang bahan langkah jakarta di akhir pekan ...,"[tentang, bahan, langkah, jakarta, di, akhir, ...",bahan langkah jakarta pekan gorengan gurih mur...
3,"5 Etiket Makan Buah Secara Formal, Ternyata Be...","Minggu, 08 Sep 2024 10:00 WIB",Jakarta - Etiket makan formal banyak dibagikan...,Makanan,Jakarta Etiket makan formal banyak dibagikan ...,jakarta etiket makan formal banyak dibagikan ...,"[jakarta, etiket, makan, formal, banyak, dibag...",jakarta etiket makan formal dibagikan pakar ta...
4,Resep Ayam Ungkep Serai yang Sedap Berbumbu Re...,"Minggu, 08 Sep 2024 09:00 WIB",Tentang Bahan Langkah Jakarta - Selain ayam go...,Makanan,Tentang Bahan Langkah Jakarta Selain ayam gor...,tentang bahan langkah jakarta selain ayam gor...,"[tentang, bahan, langkah, jakarta, selain, aya...",bahan langkah jakarta ayam goreng ayam ungkep ...


In [8]:
df.to_csv("/content/drive/My Drive/PPW/report/tugas_ppw/hasil_prepros.csv",encoding='utf8', index=False)

TF-IDF (Term Frequency-Inverse Document Frequency)

TF-IDF adalah metode statistik yang digunakan untuk mengevaluasi pentingnya suatu kata dalam sebuah dokumen relatif terhadap koleksi dokumen lainnya. TF-IDF sering digunakan dalam tugas seperti penggalian teks, penambangan informasi, dan pemodelan pembelajaran mesin berbasis teks.
Term Frequency (TF), yang menghitung seberapa sering sebuah kata muncul dalam dokumen, dan Inverse Document Frequency (IDF), yang menilai seberapa jarang kata tersebut muncul di seluruh dokumen dalam koleksi.

TF-IDF bekerja dengan memberikan bobot lebih tinggi pada kata-kata yang sering muncul dalam sebuah dokumen, tetapi jarang muncul di dokumen lain, sehingga membantu mengidentifikasi kata-kata yang paling relevan. Kata-kata umum yang muncul di banyak dokumen, seperti "dan" atau "yang," akan mendapatkan bobot rendah karena dianggap kurang informatif.

In [9]:
import pandas as pd

data = pd.read_csv("/content/drive/My Drive/PPW/report/tugas_ppw/hasil_prepros.csv", sep=",")

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Menginisialisasi TfidfVectorizer
vectorizer = TfidfVectorizer()

# Menghitung TF-IDF
tfidf_matrix = vectorizer.fit_transform(df['stopword_removal'])

In [11]:
# Mengubah hasilnya menjadi DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())
tfidf_df.head(10)

,aa,abad,abang,abc,abnormal,abuabu,acak,acar,acara,acaraki,...,yuwono,zaeni,zaitun,zaman,zat,zef,zermatt,zita,zona,zonanya
0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
1,0.0,0.0,0.061649,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
2,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.030405,0.0,0.0
3,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
4,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
5,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
6,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.028519,0.0,0.0,0.092841,0.0,0.0,0.000000,0.0,0.0
7,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
8,0.0,0.0,0.000000,0.0,0.042778,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.034814,0.0,0.0,0.000000,0.0,0.0
9,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
